In [1]:
#import libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
#finding current directory and respecifying it to where the data is stored
starting_directory = os.getcwd()
print("starting dirrectory: " + starting_directory)

new_dirrectory = os.chdir('C:/Users/maega/Documents/3000 PhD/3300_BEAT Labs/Projects/Cardiac-Amyloidosis-Multiple-Myeloma/010 Raw Data')
print('This is the new working dirrectory: ' + os.getcwd())

starting dirrectory: c:\Users\maega\Documents\3000 PhD\3300_BEAT Labs\Projects\Cardiac-Amyloidosis-Multiple-Myeloma\020 code
This is the new working dirrectory: C:\Users\maega\Documents\3000 PhD\3300_BEAT Labs\Projects\Cardiac-Amyloidosis-Multiple-Myeloma\010 Raw Data


In [4]:
#specify the files you want to import data from

# Define the path to the folder and file
folder_path = 'C:/Users/maega/Documents/3000 PhD/3300_BEAT Labs/Projects/Cardiac-Amyloidosis-Multiple-Myeloma/010 Raw Data'

#this file describes data for all patients in one spread sheet with multiple tabs

fname_input_labs = 'LIMS#10R19000024_outfile-by-IR+MCF_deID_2020-12-17.xlsx'
fname_input_echos = 'amyloid patients by IR w all manaul adjusts_deID_2020-12-20.xlsx'

outfile_ending = '.xlsx' #the file ending for the output data
outfile_common = 'data' #the base text to be added to a patient's identifier for the file name



In [ ]:
#sheet names in the original file of lab values
#deprecated at this time because of the get sheets above.
CR_Data_sheet = 'Cancer_Registery Data'
chemotx_sheet = 'ChemoTx'
hematx_sheet = 'Hematologic Tx'
immunotx_sheet =  'Immuno Tx'
radtx_sheet = 'Radiation Tx'
othertx_sheet = 'Other Tx'
labs_sheet = 'Labs'
md_notes_sheet = 'Physican_Notes'

#sheet name for echo data
echo_sheet = 'amyloid in subject'

In [ ]:
#sheet names for sheets you will make for different visualizations for patients
logical_sheet = 'logical' #sheet name for the binary logical data for a patient
ordersets_bin_sheet = 'ordersets-bin' #sheet title for the order sets in binary
ordersets_ranked_sheet = 'ordersets-ranked' #sheet name for the data indicating level of completeness of the orderset
labs_binary_sheet = "binary" #are there labs for a given date or not
labs_binary_sum_sheet = "binary-sum" #total number of labs collected on a given date
echo_raw_sheet = 'echo raw' #addition of echo data
echo_bin_sheet = 'echo bin' #was there an echo at a time point


In [5]:
#function to create a list of IDs 
def listofIDs(start_frame, column, length_from_start):
    all_IDs = start_frame[column] #pulls list of IDS including lab collection id
    ptIDs = [] #for storage of unique IDs

    length = int(length_from_start)

    i = 0
    for ID in all_IDs:
        if len(ID) > length_from_start:
            trimmed = ID[:length] #trims to the first 4 characters
            #if the first ID, add to the list
            if i == 0: 
                ptIDs.append(trimmed)
            #if not the first ID, and not equal to previous ID
            if i != 0 and trimmed != ptIDs[-1]: 
                ptIDs.append(trimmed)
        i= i+1 

    return ptIDs

In [6]:
#function for masking a dataframe 
def patientSectionOfFrame(start_frame, sheet_name, column_toMask, ptID):
    #ptID is a row parameter in the column_toMask
    df_use = start_frame #copy of the frame

    #first check if sheet has the column of interest
    data_top = list(start_frame.columns) #returns the headers as a list

    #determine of the sheet contains the column of interest
    if data_top.count(column_toMask) > 0: 
        #if true, now find if the patient is present
        searchForPt = start_frame[column_toMask].str.startswith(ptID).sum() #returns the number of instances of the patient
        if searchForPt > 0:
            #patient is present 
            #mask the data for the patient
            df_clean = df_use.dropna(subset= [column_toMask]) #removing NA 
            mask = df_clean[column_toMask].str.startswith(ptID) #masking for the rows where the patient has data
            df_sub_tosave = df_clean[mask] #output dataframe filtered for the patient
            return df_sub_tosave
        else:
            #the patient isn't present
            # return print("The patient " + ptID + " is not in " + sheet_name)
            pass
    else:
        #the column of interest isn't present
        # return print("The column " + column_toMask + " is not in the sheet " + sheet_name)
        pass



In [7]:
#importing those sheets
#idea is to automate the creation of the sheet name variables in the future
dataframe_names= ['request_df', 'CR_data', 'chemoTx_data', 'hemeTx_data', 'immunoTx_data', 'otherTx_data', 'radTx_data', 'labs_data', 'MD_notes_data', 'cerner_data']
#this became unnecessary because of the loop below

In [8]:
#need a list of patient identifiers to start
#pick a single sheet in the whole set that is likely to have everyone, such as labs 
start_df = pd.read_excel(fname_input_labs, sheet_name = 'Labs')

#get a list of unique patient IDS
ptIDs = listofIDs(start_df, 'DeID', 4)

#this is currently hard coded... 
# you could correct this by running the list of patient IDs across all the sheets and 
# then combining them, selecting unique again..

In [3]:
#collecting information about a file to use to make a list of sheet names
startfile= pd.ExcelFile('LIMS#10R19000024_outfile-by-IR+MCF_deID_2020-12-17.xlsx')

#get list of sheet names
sheet_names = startfile.sheet_names

In [9]:
#loop through by patient ID to sort through all the sheets in the file to create a patient file with their data from each sheet

#collecting information about a file to use to make a list of sheet names
startfile= pd.ExcelFile('LIMS#10R19000024_outfile-by-IR+MCF_deID_2020-12-17.xlsx')

#get list of sheet names
sheet_names = startfile.sheet_names

for ID in ptIDs:
      outfile = ID + '-' + outfile_common + outfile_ending #temp to test one patient      
      for sheet in sheet_names:
            #import the data for a specific sheet 
            df_new = pd.read_excel(fname_input_labs, sheet_name = sheet)
            df_use = df_new
            #check to see if the sheet contains information for a patient 
            pt_data = patientSectionOfFrame(df_new, sheet_name = sheet, column_toMask= 'DeID', ptID= ID)
            
            #saving if new file
            outfile_path = folder_path + '/'+outfile #outfile path

            #check to see if there is output data 
            output = isinstance(pt_data, pd.DataFrame)
            print("patient ", ID, " has data in ", sheet, " as dataframe: ", output)
            
            if output == True: #print results to a file
                  if os.path.exists(outfile_path):
                        #if old sheet
                        with pd.ExcelWriter(outfile, mode = 'a', if_sheet_exists = 'overlay') as writer:
                              pt_data.to_excel(writer, sheet_name = sheet, index = True)
                  else: 
                        #new sheet
                        with pd.ExcelWriter(outfile) as writer:
                              pt_data.to_excel(writer, sheet_name = sheet, index = True) #if new sheet
            else: #no dataframe, no data
                  pass

pt_data is a dataframe:  False
pt_data is a dataframe:  False
pt_data is a dataframe:  False
pt_data is a dataframe:  False
pt_data is a dataframe:  False
pt_data is a dataframe:  False
pt_data is a dataframe:  False
pt_data is a dataframe:  True
pt_data is a dataframe:  True
pt_data is a dataframe:  True
pt_data is a dataframe:  False
pt_data is a dataframe:  True
pt_data is a dataframe:  True
pt_data is a dataframe:  False
pt_data is a dataframe:  False
pt_data is a dataframe:  False
pt_data is a dataframe:  False
pt_data is a dataframe:  True
pt_data is a dataframe:  True
pt_data is a dataframe:  False
pt_data is a dataframe:  False
pt_data is a dataframe:  True
pt_data is a dataframe:  True
pt_data is a dataframe:  False
pt_data is a dataframe:  False
pt_data is a dataframe:  True
pt_data is a dataframe:  False
pt_data is a dataframe:  True
pt_data is a dataframe:  True
pt_data is a dataframe:  False
pt_data is a dataframe:  False
pt_data is a dataframe:  False
pt_data is a datafra

now time to collect the cardiac data. This will be slightly less smooth cause the data is only in one sheet of the echo section.

In [11]:
fname_input_echos = 'amyloid patients by IR w all manaul adjusts_deID_2020-12-20.xlsx'

#collecting information about a file to use to make a list of sheet names to check your dictionary below
echo_file= pd.ExcelFile('amyloid patients by IR w all manaul adjusts_deID_2020-12-20.xlsx')

#get list of sheet names
sheet_names_echo = echo_file.sheet_names

for ID in ptIDs:
      outfile = ID + '-' + outfile_common + outfile_ending #temp to test one patient      
      for sheet in sheet_names_echo:
            #import the data for a specific sheet 
            df_new = pd.read_excel(fname_input_echos, sheet_name = sheet)
            df_use = df_new
            #check to see if the sheet contains information for a patient 
            pt_data = patientSectionOfFrame(df_new, sheet_name = sheet, column_toMask= 'DeID', ptID= ID)
            
            #saving if new file
            outfile_path = folder_path + '/'+outfile #outfile path

            #check to see if there is output data 
            output = isinstance(pt_data, pd.DataFrame)
            print("patient ", ID, " has data in ", sheet, " as dataframe: ", output)
            
            if output == True: #print results to a file
                  if os.path.exists(outfile_path):
                        #if old sheet
                        with pd.ExcelWriter(outfile, mode = 'a', if_sheet_exists = 'overlay') as writer:
                              pt_data.to_excel(writer, sheet_name = sheet, index = True)
                  else: 
                        #new sheet
                        with pd.ExcelWriter(outfile) as writer:
                              pt_data.to_excel(writer, sheet_name = sheet, index = True) #if new sheet
            else: #no dataframe, no data
                  pass

patient  A-02  has data in  amyloid in subject  as dataframe:  True
patient  A-02  has data in  Readme  as dataframe:  False
patient  A-02  has data in  SQL  as dataframe:  False
patient  A-02  has data in  Sheet1  as dataframe:  False
patient  A-03  has data in  amyloid in subject  as dataframe:  True
patient  A-03  has data in  Readme  as dataframe:  False
patient  A-03  has data in  SQL  as dataframe:  False
patient  A-03  has data in  Sheet1  as dataframe:  False
patient  B-01  has data in  amyloid in subject  as dataframe:  True
patient  B-01  has data in  Readme  as dataframe:  False
patient  B-01  has data in  SQL  as dataframe:  False
patient  B-01  has data in  Sheet1  as dataframe:  False
patient  B-02  has data in  amyloid in subject  as dataframe:  True
patient  B-02  has data in  Readme  as dataframe:  False
patient  B-02  has data in  SQL  as dataframe:  False
patient  B-02  has data in  Sheet1  as dataframe:  False
patient  B-03  has data in  amyloid in subject  as dataf

In [ ]:
#getting the headers for the labs dataframe

problem with the below is that they don't match the new column headers, need to fetch those... 

In [ ]:
#other labels you might need 

#patient identifying information 
pt_columns = ["DeID", "RelTime", "Survival"]

#creating a dictionary for the order sets
ordersets_dict = {
    "A" : ["BUN", "Creatinine"], #liver function and nitrogen 
    "B" : ["Hemoglobin", "HemoglobinLowerLim", "Platelet"], #basic heme labs
    "C" : ["KappaFLC", "LambdaFLC", "FLCratio"], #special labs for light chains
    "D" : ["TotalUrineProtein", "UrineAlbumin"],
    "E" : ["NTproBNP", "TroponinT"], #cardio labs often ordered together
    "B2M" : ["B2M"], 
    "LDH" : ["LDH"], 
    "UricAcid": ["UricAcid"]
}

#dictionary for the markers
markers = {
    "A" : "s", #square 
    "B" : "p", #plus 
    "C" : "*", #star
    "D" : "X", #x
    "E" : "^", #triangle up
    "B2M" : "o", #circle
    "LDH" : "o", #circle
    "UricAcid": "o" #circle
}

In [ ]:
#create a way to scan headers for compatibility with the dicitonary 

#make list of headers for a sheet

#if item from list of headers starts with or contains a wording from dictionary
    #add these columns to a fresh dataframe

#OR 
#create binaries for all the data, then only plot the ones you are interested in
#do the column filtering 

In [ ]:
#create a function to create a new sheet from a given sheet 